In [108]:
import pandas

df = pandas.read_csv(
    "./3.4.csv", index_col="id", parse_dates=["reporting_start", "reporting_end"]
)
df

,name,position,department,authority,state,net,gross,reporting_start,reporting_end
id,,,,,,,,,
16311845,Adolfo Hernandez Hernandez,Fontanero,OBRAS PUBLICAS,Jaltocán,Hidalgo,4000.00,4254.00,2018-01-01,2018-06-30
16480190,ARELY SAMANTA CLEOFAS VELASCO,"AUXILIAR DE ENFERMERIA ""A""",H.G. ENRIQUE CABRERA,Secretaría de Salud,Ciudad de México,12177.86,16092.00,2018-01-01,2018-03-31
17599078,MELODY OLIMPIC GONZALEZ MONTES,POLICIA PRIMERO,SUBSECRETARIA DE OPERACION POLICIAL,Secretaría de Seguridad Ciudadana,Ciudad de México,11652.00,16030.00,2020-01-01,2020-03-31
6514612,ANGEL ALLENDE PULIDO,APOYO Y ASISTENCIA A LA EDUCACION,DIRECCIÓN GENERAL DE OPERACIONES DE SERVICIOS ...,Autoridad Educativa Federal en la Ciudad de Mé...,Federación,10180.57,2910.65,2018-07-01,2018-12-31
11927166,Yolanda Reyes Gonzalez,DIRECTOR,ACCION CIVICA,MUNICIPIO DE RINCÓN DE ROMOS,Aguascalientes,17004.40,6188.40,2019-07-01,2019-12-31
...,...,...,...,...,...,...,...,...,...
18698843,Nancy Fanny Rivera Martinez,Operativo,Sector 52,Policía Auxiliar,Ciudad de México,12119.57,13766.68,2019-04-01,2019-06-30
16658324,MONICA IGNACIA AGUILAR RODRIGUEZ,JEFE DE OFICINA,DIRECCION GENERAL DE LOS SERVICIOS ESTATALES D...,Secretaría de Educación Guerrero (SEG),Guerrero,NaN,9882.12,2018-01-01,2018-03-31
11695594,SEBASTIAN GOMEZ SANTIZ,TECNICO EN ATENCION PRIMARIA A LA SALUD,OFICINA JURISDICCIONAL (PICHUCALCO),Secretaría/Instituto de Salud,Chiapas,7978.94,20300.52,2020-01-01,2020-06-30


| What problems should we worry about?                | What can we do about these problems? |
| --------------------------------------------------- | ------------------------------------ |
| Many unavailable `name` values should be normalized | Find and replace them with `NaN`     |


In [85]:
counts = df.name.str.upper().value_counts()
counts[counts > 1]

RESERVADO RESERVADO RESERVADO                                                                                                                                                  14609
ND ND ND                                                                                                                                                                        2738
CLASIFICADO NOMBRE DEL PERSONAL ADSCRITO. VER NOTA CLASIFICADO PRIMER APELLIDO DEL PERSONAL ADSCRITO. VER NOTA CLASIFICADO SEGUNDO APELLIDO DEL PERSONAL ADSCRITO. VER NOTA     2724
VACANTE VACANTE VACANTE                                                                                                                                                         1959
ELIMINADO.NOMBRE DE SERVIDOR PÚBLICO ELIMINADO.NOMBRE DE SERVIDOR PÚBLICO ELIMINADO.NOMBRE DE SERVIDOR PÚBLICO                                                                  1857
                                                                                               

In [86]:
lengths = pandas.Series(
    counts.index.astype(str).str.len(), index=counts.index
).sort_values()
lengths[lengths > 5][-10:]

MARIA DE LOS ANGELES MARTINEZ BELTRAN MARIA DE LOS ANGELES MARTINEZ BELTRAN MARIA DE LOS ANGELES MARTINEZ BELTRAN                                                                                                                                                                   113
INFORMACIÓN CLASIFICADA COMO RESERVADA INFORMACIÓN CLASIFICADA COMO RESERVADA INFORMACIÓN CLASIFICADA COMO RESERVADA                                                                                                                                                                116
CORONEL DE LA FUERZA AÉREA ABASTECEDOR DE MATERIAL AÉREO DIPLOMADO DE ESTADO MAYOR AÉREO LUIS ALONSO GALINDO VÁZQUEZ                                                                                                                                                                116
SEGURIDAD PUBLICA Y TRANSITO MUNICIPAL SEGURIDAD PUBLICA Y TRANSITO MUNICIPAL SEGURIDAD PUBLICA Y TRANSITO MUNICIPAL                                            

In [109]:
from re import sub

df.name = df.name.apply(
    lambda field: (
        sub(r"^(.+) \1 \1 ?$", "\\1", field) if isinstance(field, str) else field
    )
)

In [111]:
from numpy import NaN

na_values = [
    r"(?i)&nbsp;",
    r"(?i)[*01-]",
    r"(?i)AAA",
    (
        r"(?i)CLASIFICADO NOMBRE DEL PERSONAL ADSCRITO\. ?VER NOTA(?: CLASIFICADO"
        r" PRIMER APELLIDO DEL PERSONAL ADSCRITO\. VER NOTA CLASIFICADO SEGUNDO"
        r" APELLIDO DEL(?: PERSONAL ADSCRITO\. VER NOTA)?)?"
    ),
    r"(?i)DIVERSOS EN REPORTE TRIMESTRAL FONE",
    r"(?i)ELIMINADO\.NOMBRE DE SERVIDOR PÚBLICO",
    r"(?i)ERROR:#N/A",
    r"(?i)INFORMACIÓN CLASIFICADA",
    r"(?i)INFORMACIÓN CLASIFICADA COMO RESERVADA",
    (
        r"(?i)INFORMACIÓN CLASIFICADA COMO RESERVADA INFORMACIÓN CLASIFICADA COMO"
        r" RESERVADA"
    ),
    r"(?i)INFORMACIÓN CONFIDENCIAL",
    (
        r"(?i)INFORMACION CONFIDENCIAL ART\. 20 PUNTO 1 Y 21 PUNTO 1 FRACCION I DE LA"
        r" LTAIPEJM"
    ),
    (
        r"(?i)INFORMACION DE CARÁCTER SENSIBLE ACUDIR A LAS OFICINAS CENTRALES PARA"
        r" INFORMACION DEALLADA"
    ),
    r"(?i)INFORMACIÓN NO DISPONIBLE\. VER NOTA\.",
    r"(?i)INFORMACION RESERVADA",
    r"(?i)INFORMACIÓN RESERVADA",
    r"(?i)INFORMACION RESERVADA \(\*\)",
    r"(?i)N/?[AD]",
    r"(?i)ND= NO HAY DATO VER CAMPO DE NOTA\.",
    r"(?i)NO DATO",
    r"(?i)NULL",
    r"(?i)PERSONAL ADSCRITO\. VER NOTA",
    r"(?i)POR DESIGNAR",
    r"(?i)RESERVAD[AO]",
    r"(?i)SIN ESPECIFICAR",
    r"(?i)VACANTE",
    r"(?i)VÉASE NOTA",
    r"(?i)X(?:XX(?:XXX)?)?",
]

df.name.replace(na_values, NaN, inplace=True, regex=True)

In [112]:
counts = df.name.str.upper().value_counts()
lengths = pandas.Series(
    counts.index.astype(str).str.len(), index=counts.index
).sort_values()
lengths[lengths > 5].index[-10:]

Index(['TENIENTE CORONEL DE JUSTICIA MILITAR Y LICENCIADO ALBERTO SANTOME DÍAZ',
       'CORONEL INGENIERO EN COMPUTACIÓN E INFORMÁTICA VÍCTOR JIMÉNEZ DOMÍNGUEZ',
       'GENERAL BRIGADIER DIPLOMADO DE ESTADO MAYOR JORGE ARTURO OLIVERA OROZCO',
       'TENIENTE CORONEL DE JUSTICIA MILITAR Y LICENCIADO FRANCISCO RUBIO CASTRO',
       'MARIA CECILIA DEL SAGRADO CORAZON DE JESUS GONZALEZ DE LA VEGA MIER Y TERAN',
       'TENIENTE CORONEL DE LA FUERZA AÉREA ARMAMENTO AÉREO ENRIQUE ROSALES RODRÍGUEZ',
       'CAPITAN PRIMERO DE INFANTERÍA DIPLOMADO DE ESTADO MAYOR ARNULFO ESPINOZA PÉREZ',
       'GENERAL DE BRIGADA DE JUSTICIA MILITAR Y LICENCIADO JESÚS GABRIEL LÓPEZ BENÍTEZ',
       'TENIENTE CORONEL DE LA FUERA AÉREA ABASTECEDOR DE MATERIAL AÉREO ARTURO LOPEZ NUÑEZ',
       'TENIENTE CORONEL DE ARTILLERÍA DIPLOMADO DE ESTADO MAYOR MARCO ANTONIO MENCHACA MARIN'],
      dtype='object')

In [113]:
counts[counts > 1][:30]

MARIO GARCIA LOPEZ                  66
YESICA MARISOL RUBIO GOMEZ          65
GERARDO SANCHEZ CASTILLO            61
JUAN JOSE GARCIA ORTIZ              61
LAURA NERI SANCHEZ                  59
JOSE RAMON TOVAR FLORES             57
CESAR GUADALUPE RUIZ VALENCIA       54
DANIEL MACIAS MORA                  52
JORGE HOUSTON RODRIGUEZ MATA        52
CHRISTIAN OLIVER GONZALEZ ORTIZ     52
JAVIER ARELLANO ALBA                51
FRIDA VALENCIA MARTINEZ             49
MARIA DEL ROSARIO MORENO PEREZ      49
JORGE ARTURO LOPEZ REYES            49
MARCELA IVETH MONTAÑEZ LUEVANO      47
JAVIER SALAS GONZALEZ               47
RICARDO MARTINEZ MARTINEZ           46
ADA NELLY LOPEZ MATA                46
JULIAN RENE BARAJAS OLMOS           46
CLAUDIA MARICELA AGUILERA FRANCO    45
DAINZU DE VELASCO ALCALA            45
IRMA NERI SANCHEZ                   45
JAIME LUIS RODRIGUEZ MORENO         45
JORGE ARTURO SILVA CERVANTES        45
CESAR VLADIMIR ALCALA MATA          45
MARIBEL RODRIGUEZ CAMPOS 

In [114]:
df

,name,position,department,authority,state,net,gross,reporting_start,reporting_end
id,,,,,,,,,
16311845,NaN,Fontanero,OBRAS PUBLICAS,Jaltocán,Hidalgo,4000.00,4254.00,2018-01-01,2018-06-30
16480190,ARELY SAMANTA CLEOFAS VELASCO,"AUXILIAR DE ENFERMERIA ""A""",H.G. ENRIQUE CABRERA,Secretaría de Salud,Ciudad de México,12177.86,16092.00,2018-01-01,2018-03-31
17599078,MELODY OLIMPIC GONZALEZ MONTES,POLICIA PRIMERO,SUBSECRETARIA DE OPERACION POLICIAL,Secretaría de Seguridad Ciudadana,Ciudad de México,11652.00,16030.00,2020-01-01,2020-03-31
6514612,NaN,APOYO Y ASISTENCIA A LA EDUCACION,DIRECCIÓN GENERAL DE OPERACIONES DE SERVICIOS ...,Autoridad Educativa Federal en la Ciudad de Mé...,Federación,10180.57,2910.65,2018-07-01,2018-12-31
11927166,NaN,DIRECTOR,ACCION CIVICA,MUNICIPIO DE RINCÓN DE ROMOS,Aguascalientes,17004.40,6188.40,2019-07-01,2019-12-31
...,...,...,...,...,...,...,...,...,...
18698843,NaN,Operativo,Sector 52,Policía Auxiliar,Ciudad de México,12119.57,13766.68,2019-04-01,2019-06-30
16658324,NaN,JEFE DE OFICINA,DIRECCION GENERAL DE LOS SERVICIOS ESTATALES D...,Secretaría de Educación Guerrero (SEG),Guerrero,NaN,9882.12,2018-01-01,2018-03-31
11695594,SEBASTIAN GOMEZ SANTIZ,TECNICO EN ATENCION PRIMARIA A LA SALUD,OFICINA JURISDICCIONAL (PICHUCALCO),Secretaría/Instituto de Salud,Chiapas,7978.94,20300.52,2020-01-01,2020-06-30


In [116]:
df.name = df.name.apply(
    lambda field: (
        sub(r" ?N\A ?", " ", field).strip() if isinstance(field, str) else field
    )
)

In [117]:
df.to_csv("./3.5.csv")